In [227]:
import subprocess
import multiprocessing
import os
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
import time
from scipy.stats import ttest_rel

In [2]:
neighbourhoods = ["--transpose", "--exchange", "--insert", "--tei", "--tie"]
def run_fssp(instance, init=0, improvement=0, neighbourhood=0):
    try:
        init = "--random-init" if not init else "--srz"
        improvement = "--first" if not improvement else "--best"
        neighbourhood = neighbourhoods[neighbourhood]
        command = "./fssp " + " ".join([instance, init, improvement, neighbourhood])
        beg_time = time.time()
        output = subprocess.check_output(command, shell=True, stderr=subprocess.DEVNULL)
        solution, score = output.decode('utf-8').split('\n')[:2]
        return command, [int(x) for x in solution.split(" ") if x], int(score), time.time() - beg_time
    except Exception as e:
        print(e)
        return command, [], 0


In [3]:
def run_instance(instance):
    res = []
    for init in [0, 1]:
        for improvement in [0, 1]:
            for neighbourhood in [0, 1, 2]:
                res.append(run_fssp(instance, init, improvement, neighbourhood))
        for neighbourhood in [3, 4]:
            res.append(run_fssp(instance, init, 0, neighbourhood))
    return res


In [4]:
def all_instances_names():
    for filename in os.listdir("../instances"):
        if filename != "bestSolutions.txt":
            yield os.path.join("../instances/", filename)

In [5]:
instances = list(all_instances_names())
pool = multiprocessing.Pool(multiprocessing.cpu_count())
res = list(tqdm(pool.imap_unordered(run_instance, instances), total=len(instances)))        

In [6]:
import itertools
a = list(itertools.chain(*res))
len(a)

960

def to_nikita(a):
    times = {}
    for x in a:
        times[x[0].replace("../", "").replace("--tie", "--vnd-tie").replace("--tei", "--vnd-tei")] = x[2]
    return times

times = to_nikita(a)

In [8]:
processed = [
    {
        "instance": s.split()[1].split("/")[-1],
        "n_jobs": len(sol),
        "init": "srz" if "srz" in s else "random",
        "pivot": "best" if "best" in s else "first",
        "neighbourhood": "exchange" if "exchange" in s else "transpose" if "transpose" in s else "insert" if "insert"in s else "tie" if "tie" in s else "tei",
        "score": score,
        "time": time
    }
    for s, sol, score, time in a
]
df = pd.DataFrame(processed)

In [9]:
df

,init,instance,n_jobs,neighbourhood,pivot,score,time
0,random,50_20_28,50,transpose,first,725848,0.045286
1,random,50_20_28,50,exchange,first,566459,0.790496
2,random,50_20_28,50,insert,first,575218,0.736156
3,random,50_20_28,50,transpose,best,724189,0.016100
4,random,50_20_28,50,exchange,best,572886,0.181353
5,random,50_20_28,50,insert,best,583491,0.257569
6,random,50_20_28,50,tei,first,559822,0.080217
7,random,50_20_28,50,tie,first,561436,0.052149
8,srz,50_20_28,50,transpose,first,565683,0.014031
9,srz,50_20_28,50,exchange,first,565197,0.047408


In [10]:
best = pd.read_csv("../instances/bestSolutions.txt")
best.columns = ["instance", "best_solution"]
best.instance = best.instance.str.strip()
best.best_solution = best.best_solution.astype(int)
best.sample()

,instance,best_solution
32,50_20_03,592745


In [11]:
merged = pd.merge(df, best, how='left', on='instance')
merged.sample()

,init,instance,n_jobs,neighbourhood,pivot,score,time,best_solution
145,random,100_20_22,100,exchange,first,1781409,17.523705,1740800


In [12]:
merged['percentage_deviation'] = 100 * (merged.score - merged.best_solution) / merged.best_solution

In [13]:
exercise1 = merged[(merged.neighbourhood != "tei") & (merged.neighbourhood != "tie")]
print(len(exercise1))
exercise1.sample()

720


,init,instance,n_jobs,neighbourhood,pivot,score,time,best_solution,percentage_deviation
257,random,50_20_16,50,exchange,first,557831,0.767723,551243,1.195117


# Exercise 1.1

In [20]:
exercise1_groupby_algos = exercise1.groupby(['init', 'pivot', 'neighbourhood', 'n_jobs'])
exercise1_groupby_algos.mean()[['percentage_deviation']]

percentage_deviation
init   pivot neighbourhood n_jobs                      
random best  exchange      50                  4.385601
                           100                 4.667283
             insert        50                  8.170156
                           100                10.903816
             transpose     50                 32.736274
                           100                42.109242
       first exchange      50                  1.965092
                           100                 1.715059
             insert        50                  6.265681
                           100                 7.724756
             transpose     50                 31.101046
                           100                41.205408
srz    best  exchange      50                  2.787518
                           100                 3.432429
             insert        50                  2.811639
                           100                 3.511336
             transpose     50                  3.701844
                           100                 4.587164
       first exchange      50                  2.858113
                           100                 3.119183
             insert        50                  2.669091
                           100                 3.202606
             transpose     50                  3.680069
                           100                 4.580643

In [21]:
exercise1_groupby_algos.mean()[['time']]

time
init   pivot neighbourhood n_jobs           
random best  exchange      50       0.226340
                           100      3.609992
             insert        50       0.264383
                           100      4.113701
             transpose     50       0.017199
                           100      0.081026
       first exchange      50       0.702442
                           100     16.405160
             insert        50       0.894131
                           100     21.310241
             transpose     50       0.027533
                           100      0.091783
srz    best  exchange      50       0.063340
                           100      0.820232
             insert        50       0.064307
                           100      0.800185
             transpose     50       0.016418
                           100      0.043156
       first exchange      50       0.057423
                           100      1.115112
             insert        50       0.077194
                           100      1.148529
             transpose     50       0.014516
                           100      0.041757

# Exercise 1.2 (ttest)

## Exercise 1.2.(i) (which initial solution is preferable)

In [312]:
exercise1_algos = pd.DataFrame(exercise1.groupby(['init', 'pivot', 'neighbourhood']).groups).T
exercise1_algos_scores = exercise1_algos.applymap(lambda x: exercise1.loc[x].percentage_deviation)
exercise1_algos_scores['values'] = pd.concat([exercise1_algos_scores[x] for x in exercise1_algos_scores.columns], axis=1).values.tolist()
exercise1_algos_scores = exercise1_algos_scores.drop(exercise1_algos_scores.columns.difference(['values']), axis=1)
exercise1_algos_times = exercise1_algos.applymap(lambda x: exercise1.loc[x].time)
exercise1_algos_times['values'] = pd.concat([exercise1_algos_times[x] for x in exercise1_algos_times.columns], axis=1).values.tolist()
exercise1_algos_times = exercise1_algos_times.drop(exercise1_algos_times.columns.difference(['values']), axis=1)

def student_two_columns(df, left, right):
    print(left, right)
    df = df['values'][[left, right]]
    return df.apply(lambda x: ttest_rel(x[left], x[right]), axis=1) \
             .rename(columns={left: 't-statistic', right: 'p-value'})

In [313]:
student_two_columns(exercise1_algos_scores.unstack(level=0), 'random', 'srz')

random srz


t-statistic       p-value
best  exchange     10.476357  4.369954e-15
      insert       19.474311  2.261227e-27
      transpose    37.461813  7.836271e-43
first exchange     -9.202500  5.260806e-13
      insert       16.750731  4.282623e-24
      transpose    34.188782  1.364952e-40

## Exercise 1.2.(ii) (which pivoting rule generates better quality solutions and which is faster)

In [314]:
student_two_columns(exercise1_algos_scores.unstack(level=1), 'best', 'first')

best first


t-statistic       p-value
random exchange     20.873300  6.287508e-29
       insert        9.874594  4.116715e-14
       transpose     7.331677  7.418797e-10
srz    exchange      1.496189  1.399349e-01
       insert        2.801594  6.866970e-03
       transpose     1.079051  2.849567e-01

In [238]:
exercise1_algos_scores.unstack(level=1).applymap(np.mean)

values           
                       best      first
random exchange    4.526442   1.840076
       insert      9.536986   6.995218
       transpose  37.422758  36.153227
srz    exchange    3.109973   2.988648
       insert      3.161487   2.935848
       transpose   4.144504   4.130356

In [315]:
student_two_columns(exercise1_algos_times.unstack(level=1), 'best', 'first')

best first


t-statistic       p-value
random exchange     -8.075774  4.074129e-11
       insert       -8.012502  5.211814e-11
       transpose    -5.516157  8.082804e-07
srz    exchange     -2.859979  5.849614e-03
       insert       -4.329634  5.869584e-05
       transpose     3.104344  2.927752e-03

In [236]:
exercise1_algos_times.unstack(level=1).applymap(np.mean)

values           
                      best      first
random exchange   1.918166   8.553801
       insert     2.189042  11.102186
       transpose  0.049112   0.059658
srz    exchange   0.441786   0.586267
       insert     0.432246   0.612862
       transpose  0.029787   0.028137

## Exercise 1.2.(iii) (which neighborhood generates better quality solution and what computation time is required to reach local optima)

In [309]:
student_two_columns(exercise1_algos_scores.unstack(level=2), 'exchange', 'insert')

exchange insert


t-statistic       p-value
random best    -13.951116  2.432585e-20
       first   -21.633866  9.676010e-30
srz    best     -0.611820  5.430072e-01
       first     0.511523  6.108939e-01

In [310]:
student_two_columns(exercise1_algos_scores.unstack(level=2), 'exchange', 'transpose')

exchange transpose


t-statistic       p-value
random best    -36.015470  7.267839e-42
       first   -34.908654  4.226097e-41
srz    best     -9.571960  1.289837e-13
       first   -10.442561  4.951419e-15

In [311]:
student_two_columns(exercise1_algos_scores.unstack(level=2), 'insert', 'transpose')

insert transpose


t-statistic       p-value
random best    -35.444221  1.791755e-41
       first   -32.976053  1.035748e-39
srz    best    -10.546375  3.374830e-15
       first   -11.682677  5.520169e-17

In [234]:
exercise1_algos_scores.unstack(level=2).applymap(np.mean)

values                     
              exchange    insert  transpose
random best   4.526442  9.536986  37.422758
       first  1.840076  6.995218  36.153227
srz    best   3.109973  3.161487   4.144504
       first  2.988648  2.935848   4.130356

In [316]:
student_two_columns(exercise1_algos_times.unstack(level=2), 'exchange', 'insert')

exchange insert


t-statistic       p-value
random best     -4.278296  7.000695e-05
       first    -6.149409  7.296485e-08
srz    best      0.320772  7.495166e-01
       first    -0.483018  6.308702e-01

In [317]:
student_two_columns(exercise1_algos_times.unstack(level=2), 'exchange', 'transpose')

exchange transpose


t-statistic       p-value
random best      8.549876  6.472278e-12
       first     8.206635  2.449287e-11
srz    best      7.516353  3.607924e-10
       first     6.801021  5.870356e-09

In [319]:
student_two_columns(exercise1_algos_times.unstack(level=2), 'insert', 'transpose')

insert transpose


t-statistic       p-value
random best      8.479240  8.507025e-12
       first     8.147806  3.078581e-11
srz    best      7.800300  1.191504e-10
       first     7.771280  1.334274e-10

In [233]:
exercise1_algos_times.unstack(level=2).applymap(np.mean)

values                     
              exchange     insert transpose
random best   1.918166   2.189042  0.049112
       first  8.553801  11.102186  0.059658
srz    best   0.441786   0.432246  0.029787
       first  0.586267   0.612862  0.028137

# Exercise 2

In [284]:
exercise2 = merged[(merged.neighbourhood == "tei") | (merged.neighbourhood == "tie")]
print(len(exercise2))
exercise2.sample()

240


,init,instance,neighbourhood,pivot,score,time,best_solution,percentage_deviation
270,srz,100_20_13,tei,first,1749269,2.024878,1712830,2.127415


In [285]:
exercise2_groupby_algos = exercise2.groupby(['init', 'pivot', 'neighbourhood'])
exercise2_groupby_algos.mean()[['percentage_deviation']]

percentage_deviation
init   pivot neighbourhood                      
random first tei                        2.300083
             tie                        4.771551
srz    first tei                        2.550408
             tie                        2.745331